In [2]:
%matplotlib inline
from randimage import get_random_image, show_array
import random
import pandas as pd
from sklearn.datasets import load_digits
import cv2
import numpy as np
from PIL import Image
from keras import utils
# from keras.utils.np_utils import to_categorical 
from keras import backend as K
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten,Activation
from keras.models import Sequential, Model
# from tensorflow.keras.optimizers import RMSprop
from keras.optimizers import RMSprop
# from tensorflow.keras import optimizers
from keras import optimizers
import matplotlib.pyplot as plt 
from keras import callbacks
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.models import Model,load_model, model_from_json
import tensorflow as tf
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization

#enable eager execution in tensorflow
tf.config.run_functions_eagerly(True)
to_categorical = utils.to_categorical

In [1]:
%matplotlib inline
from randimage import get_random_image, show_array
import random
import pandas as pd
from sklearn.datasets import load_digits
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras import utils
# from tensorflow.keras.utils.np_utils import to_categorical 
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import RMSprop
# from tensorflow.keras import optimizers
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt 
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.models import Model, load_model, model_from_json
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization

# Enable eager execution in tensorflow
tf.config.run_functions_eagerly(True)
to_categorical = utils.to_categorical


2023-08-17 22:37:20.828723: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-17 22:37:20.855435: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-17 22:37:20.856148: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 22:37:21.525501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
digits_ = load_digits()
target_ = digits_['target']
target_ = train_labels = to_categorical(target_, num_classes=10)
digits = digits_['data'].reshape(-1,8,8)

digits_resize = np.zeros((len(digits),32,32))

for x,y in enumerate(digits):
    digits_resize[x] = (cv2.resize(y, dsize=(32,32), interpolation=cv2.INTER_CUBIC)+1)/2

In [19]:
def build_base_network(input_shape=(32,32,1)):
    
    model = Sequential()
    model.add(Convolution2D(16, (8,8), strides=(1,1),activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(32, (4,4),strides=(1,1), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(10,activation='softmax'))
    return model

model = build_base_network()
rms = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)#RMSprop()
rms = RMSprop()
earlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              restore_best_weights=True)
callback_early_stop_reduceLROnPlateau=[earlyStopping]

model.compile(loss='categorical_crossentropy', optimizer=rms,metrics="accuracy")
model.summary()

batch_size=32
model.fit(digits_resize.reshape(-1,32,32,1), target_, validation_split=.20,batch_size= batch_size, verbose=1, epochs=2, callbacks=callback_early_stop_reduceLROnPlateau)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 25, 25, 16)        1040      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 9, 9, 32)          8224      
                                                                 
 dropout_3 (Dropout)         (None, 9, 9, 32)          0         
                                                                 
 flatten_1 (Flatten)         (None, 2592)              0         
                                                                 
 dense_3 (Dense)             (None, 512)               1327616   
                                                      

/home/emon/anaconda3/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


45/45 [==============================] - 2s 44ms/step - loss: 1.5027 - accuracy: 0.4906 - val_loss: 0.5057 - val_accuracy: 0.8639
Epoch 2/2
45/45 [==============================] - 2s 42ms/step - loss: 0.5410 - accuracy: 0.8212 - val_loss: 0.3738 - val_accuracy: 0.8833


# Generating dummy images for one shot

In [5]:
one = []
zero = []

img_size = (32,32)

for x in range(200):
    
    img = get_random_image(img_size)  
    
    #picking random a,b,c,d coordinates for plotting rectangle
    a,b = random.randrange(0,img_size[0]/4),random.randrange(0,img_size[0]/4)
    c,d = random.randrange(img_size[0]/2,img_size[0]),random.randrange(img_size[0]/2,img_size[0])
    
    value = random.sample([True,False],1)[0]
    if value==False:
        #plotting rectangle 
        img[a:c,b:d,0] = 25
        img[a:c,b:d,1] = 25
        img[a:c,b:d,2] = 25
        #convert RGB image to black & white
        img = np.asarray(Image.fromarray((img*255).astype(np.uint8)).convert('L'))/255
        one.append(img)
    else:
        img = np.asarray(Image.fromarray((img*255).astype(np.uint8)).convert('L'))/255
        zero.append(img)

/tmp/ipykernel_23205/4108693065.py:11: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  a,b = random.randrange(0,img_size[0]/4),random.randrange(0,img_size[0]/4)
/tmp/ipykernel_23205/4108693065.py:12: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  c,d = random.randrange(img_size[0]/2,img_size[0]),random.randrange(img_size[0]/2,img_size[0])


In [20]:
def euclidean_distance(vects):
    #euclidean distance, output for Siamese network
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [14]:
total_sample_size = 50
test_sample_size = 200
dim1,dim2 = 32,32
count = 0

x_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])
y = np.zeros([total_sample_size,1])

x_pair_test = np.zeros([test_sample_size, 2, 1, dim1, dim2])
y_test = np.zeros([test_sample_size,1])

for x in range(total_sample_size):
    value = random.sample([True,False],1)[0]
    if value:
            pair = random.choices(one, k=2)
            x_pair[x,0,0,:,:] = pair[0]
            x_pair[x,1,0,:,:] = pair[1]
            #setting label=1 for similar images
            y[x] = 1
    
    else:
            x_pair[x,0,0,:,:] = random.choices(one, k=1)[0]
            x_pair[x,1,0,:,:] = random.choices(zero, k=1)[0]
            #setting label=0 for dissimilar images
            y[x] = 0


for x in range(test_sample_size):
    value = random.sample([True,False],1)[0]
    if value:
            pair = random.choices(one, k=2)
            x_pair_test[x,0,0,:,:] = pair[0]
            x_pair_test[x,1,0,:,:] = pair[1]
            y_test[x] = 1
    
    else:
            x_pair_test[x,0,0,:,:] = random.choices(one, k=1)[0]
            x_pair_test[x,1,0,:,:] = random.choices(zero, k=1)[0]
            y_test[x] = 0
x_pair.shape, x_pair_test.shape, y.shape, y_test.shape

((50, 2, 1, 32, 32), (200, 2, 1, 32, 32), (50, 1), (200, 1))

In [21]:
#remove the final output layer from the CNN we added for MNIST Classification
model2= Model(inputs=model.input, outputs=model.layers[-2].output)

input_dim = x_pair.shape[3:]+ tuple([1])

img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)

feat_vecs_a = model2(img_a)
feat_vecs_b = model2(img_b)
# print(K.int_shape(feat_vecs_a))
#Siamese output using utility functions declared above
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [9]:
rms = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)#RMSprop()
earlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1,
                              restore_best_weights=True)
callback_early_stop_reduceLROnPlateau=[earlyStopping]

model = Model(inputs=[img_a, img_b], outputs=distance)
model.compile(loss=contrastive_loss, optimizer=rms,metrics=[accuracy])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 1)]          0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 32, 32, 1)]          0         []                            
                                                                                                  
 model (Functional)          (None, 256)                  1468208   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 lambda (Lambda)             (None, 1)                    0         ['model[0][0]',         

In [22]:
img1 = x_pair[:, 0].reshape(-1,32,32,1)
img2 = x_pair[:, 1].reshape(-1,32,32,1)

img3 = x_pair_test[:, 0].reshape(-1,32,32,1)
img4 = x_pair_test[:, 1].reshape(-1,32,32,1)
img1.shape, img2.shape, img3.shape, img4.shape, y.shape, y_test.shape

((50, 32, 32, 1),
 (50, 32, 32, 1),
 (200, 32, 32, 1),
 (200, 32, 32, 1),
 (50, 1),
 (200, 1))

In [10]:
img1 = x_pair[:, 0].reshape(-1,32,32,1)
img2 = x_pair[:, 1].reshape(-1,32,32,1)

img3 = x_pair_test[:, 0].reshape(-1,32,32,1)
img4 = x_pair_test[:, 1].reshape(-1,32,32,1)

batch_size = 8
history = model.fit([img1, img2], y, validation_data=([img3,img4],y_test),
      batch_size= batch_size, verbose=1, epochs=10, callbacks=callback_early_stop_reduceLROnPlateau)

model.save_weights('model_weights.h5')
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())
print('saved')

Epoch 1/10


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [15]:
print(y_test.shape, y.shape)

(200, 1) (50, 1)
